# Import Library

In [3]:
import ortools
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

In [4]:
import time
import random as rd
from math import ceil
from math import factorial
def comb(x, y):
  return factorial(x)/(factorial(y) * factorial(x - y))
from array import *
import numpy as np
from itertools import combinations

## Data Generator

In [2]:
def genData(N, M, attendantRange, capacityRange, K):
    ''' Assume N, M, *attendantRange, and *capacityRange are positive integers.
    Assume attendantRange and capacityRange are 2-element lists or tuples.
    Assume attendantRange[0] <= attendantRange[1] <= capacityRange[1] and capacityRange[0] <= capacityRange[1].
    Assume K is a natural number at most N choose 2.
    
    Randomly generate N courses, M exam rooms, and K conflicts for exam scheduling algorithms 
    such that the number of candidates attending any exam is between attendantRange[0] and attendantRange[1]
    while the capacity of any exam hall is between capacityRange[0] and capacityRange[1].
    
    Write data into a text file.'''
    
    assert False not in [arg > 0 for arg in (N, M, *attendantRange, *capacityRange)], 'N, M, *Range, and *capacityRange should be positive integers.'
    assert K >= 0 and K <= comb(N, 2), 'K should be a natural number at most N choose 2.'

    attendants = [str(rd.randint(attendantRange[0], attendantRange[1])) for i in range(N)]
    #generate a number of large halls which can occupy all candidates of any exam and a number of halls which cannot
    numLargeRooms = rd.randint(1, M)
    smallRooms = [str(rd.randint(capacityRange[0], attendantRange[1])) for i in range(M - numLargeRooms)]
    largeRooms = [str(rd.randint(attendantRange[1], capacityRange[1])) for i in range(numLargeRooms)]
    capacities = smallRooms + largeRooms
    rd.shuffle(capacities)
    #generate all possible pairs of exams with common candidates and pick K random pairs
    conflicts = list(combinations(range(1, N + 1), 2))
    rd.shuffle(conflicts)
    conflicts = [[str(i), str(j)] for i, j in conflicts[:K]]
    for pair in conflicts:
        rd.shuffle(pair)
    
    filename = f'C:/Users/Admin/Desktop/Capstone-Project-Optimization/Project Optimization/data/data-N{N}-M{M}-d-{attendantRange[0]}-{attendantRange[1]}-c-{capacityRange[0]}-{capacityRange[1]}-K{K}.txt'
    with open(filename, 'w') as file:
        #line 1: N 
        file.write(str(N))
        #line 2: d1, d2, ..., dN
        file.write('\n' + ' '.join(attendants))
        #line 3: M
        file.write('\n' + str(M))
        #line 4: c1, c2, ..., cM'''
        file.write('\n' + ' '.join(capacities))
        #line 5: K
        file.write('\n' + str(K))
        #lines from 6 to 5 + K: pairs of exams with common candidates
        for pair in conflicts:
            file.write('\n' + ' '.join(pair))
    
    return filename

In [3]:
# if __name__ == '__main__':
#     n = int(input('Number of exams: N = '))
#     m = int(input('Number of examination rooms: M = '))
#     d = [int(i) for i in input('min and max number of candidates for any exam: ').split()]
#     c = [int(i) for i in input('min and max capacity of any exam room: ').split()]
#     k = int(input('Number of pairs of exams with common candidates: K = '))
#     print('Check ' + genData(n, m, d, c, k) + '.')

## Data Reader

In [1]:
filename = 'C:/Users/Admin/Desktop/Capstone-Project-Optimization/Project Optimization/data/data-N200-M25-d-20-60-c-22-65-K500.txt'
def readData(filename):
  with open(filename) as f:
    content = [[int(j) for j in i.split()] for i in f.read().splitlines()]
  N, d, M, c, K = content[0][0], content[1], content[2][0], content[3], content[4][0]
  p = [[content[5 + i][0] - 1, content[5 + i][1] - 1] for i in range(K)]
  print(f'N = {N}', f'd = {d}', f'M = {M}', f'c = {c}', f'K = {K}', f'p = {p}', sep = '\n')
  return N, d, M, c, K, p
N, d, M, c, K, p = readData(filename)

N = 200
d = [22, 59, 56, 35, 39, 33, 51, 29, 59, 51, 41, 38, 51, 56, 46, 55, 50, 25, 42, 22, 47, 59, 44, 51, 42, 25, 20, 22, 37, 42, 37, 48, 43, 58, 20, 29, 22, 49, 55, 44, 57, 54, 35, 45, 28, 33, 42, 43, 24, 21, 21, 24, 46, 31, 31, 24, 33, 35, 26, 54, 50, 36, 36, 20, 29, 23, 30, 48, 43, 28, 47, 51, 37, 25, 46, 57, 28, 28, 23, 57, 29, 42, 41, 21, 48, 45, 40, 52, 46, 27, 34, 57, 33, 54, 21, 35, 54, 45, 27, 54, 57, 46, 22, 40, 25, 53, 31, 54, 37, 46, 22, 55, 47, 41, 56, 28, 41, 27, 54, 40, 26, 23, 42, 48, 59, 20, 58, 27, 51, 22, 58, 36, 59, 21, 44, 58, 38, 27, 50, 56, 60, 24, 55, 57, 23, 46, 20, 60, 22, 35, 51, 52, 21, 30, 33, 54, 34, 21, 57, 20, 60, 27, 60, 30, 39, 38, 20, 41, 28, 28, 36, 43, 35, 29, 48, 42, 21, 32, 23, 48, 39, 25, 46, 33, 30, 38, 50, 23, 46, 58, 57, 27, 32, 21, 52, 54, 55, 25, 36, 21]
M = 25
c = [60, 63, 61, 39, 63, 63, 65, 60, 26, 65, 64, 61, 65, 65, 62, 63, 64, 30, 61, 55, 63, 64, 61, 64, 51]
K = 500
p = [[177, 154], [129, 151], [7, 199], [52, 0], [144, 4], [183, 21]

## Solution Printer

In [5]:
def printSolution():
  # Print the objective value
  print(f'The minimum number periods needed: {obj_value}, equivalent to: {ceil(obj_value / 4)} days.')
  print('------------------')
  # Print the solution matrix
  for i in range(obj_value):
    print(f'Period {i + 1}')
    for j in range(M):
      if solution_matrix[i][j] != -1:
        print(f'\tRoom {j + 1}: Course {solution_matrix[i][j] + 1}, attendant {d[solution_matrix[i][j]]}, capacity {c[j]}.')

# Algorithms

## Mixed Integer Programming

In [43]:
mip_solver = pywraplp.Solver.CreateSolver('SCIP')

# Infinity
INF = mip_solver.infinity()

# Define variables

# Variable x[i][j][k]
x = [[[mip_solver.IntVar(0, 1, f'x[{i}][{j}][{k}]') for i in range(N)] for j in range(M)] for k in range(N)]

# Variable y
y = mip_solver.IntVar(0, N - 1, 'y')

# Define constraints

# Constraint 1: Pairs of conflicting courses may not be put in the same period
for i in range(K):
  u, v = p[i][0], p[i][1]
  for k in range(N):
    constraint = mip_solver.Constraint(0, 1)
    for j1 in range(M):
      for j2 in range(M):
        if j1 != j2:
          constraint.SetCoefficient(x[u][j1][k], 1)
          constraint.SetCoefficient(x[v][j2][k], 1)

# Constraint 2: An course room may be assigned at most one course in a period
for j in range(M):
  for k in range(N):
    constraint = mip_solver.Constraint(0, 1)
    for i in range(N):
      constraint.SetCoefficient(x[i][j][k], 1)

# Constraint 3: The number of periods (k.x[i,j,k] - y <= 0)
for i in range(N):
  for j in range(M):
    for k in range(N):
      constraint = mip_solver.Constraint(-INF, 0)
      constraint.SetCoefficient(y, -1)
      constraint.SetCoefficient(x[i][j][k], k)

# Constraint 4: A course may be conducted at most one time in an course room
for i in range(N):
  constraint = mip_solver.Constraint(1, 1)
  for j in range(M):
    for k in range(N):
      constraint.SetCoefficient(x[i][j][k], 1)

# Constraint 5: A course n_i must be put into a room m_j with capacity c(j)
for i in range(N):
  for j in range(M):
    constraint = mip_solver.Constraint(0, c[j])
    for k in range(N):
      constraint.SetCoefficient(x[i][j][k], d[i])

# Define objective
obj = mip_solver.Objective()
obj.SetCoefficient(y, 1)
obj.SetMinimization()

mip_solver.SetTimeLimit(30000)

# Solve and count elapsed time
start_time = time.time()
status = mip_solver.Solve()
end_time = time.time()

# Print solution
if status == mip_solver.OPTIMAL or status == mip_solver.FEASIBLE:
  obj_value = int(obj.Value() + 1)
  solution_matrix = []
  for i in range(obj_value):
    solution_matrix.append([-1 for _ in range(M)])
  for k in range(obj_value):
    for j in range(M):
      for i in range(N):
        if x[i][j][k].solution_value() == 1:
          solution_matrix[k][j] = i
  printSolution()
else:
  print('Not found solution.')
print('------------------')

print(f'Used time: {1000*(end_time - start_time)} milliseconds')

Not found solution.
------------------
Used time: 34280.587911605835 milliseconds


## Constraint Programming

In [44]:
# Initiation
model = cp_model.CpModel()

# Variable x[i]: period of course ni
x = [model.NewIntVar(1, N, f'x[{i}]') for i in range(N)]

# Variable y[i][j]: whether course ni takes room  j or not
y = [[model.NewIntVar(0, 1, f'y[{i}][{j}]') for j in range(M)] for i in range(N)]

# Define constraints
# Constraint 1: Pairs of conflicting courses may not be put in the same period
for pair in p:
  model.Add(x[pair[0]] != x[pair[1]])

# Constraint 2: An course room  is assigned at most one course in a period
for i in range(N):
  model.Add(sum(y[i]) == 1)

# Constraint 3: Courses with same period cannot use the same room 
for j in range(M):
  for i1 in range(N - 1):
    for i2 in range(i1 + 1, N):
      b = model.NewBoolVar(f'b[{j}][{i1}][{i2}]')
      model.Add(y[i1][j] + y[i2][j] <= 1).OnlyEnforceIf(b)
      model.Add(x[i1] == x[i2]).OnlyEnforceIf(b)
      model.Add(x[i1] != x[i2]).OnlyEnforceIf(b.Not())
      
# Constraint 4: The attendance of course n_i must be smaller than capacity of room  
for i in range(N):
  model.Add(sum([y[i][j] * c[j] for j in range(M)]) >= d[i])

# Objective
cp_ob = model.NewIntVar(1, N, 'ob')
model.AddMaxEquality(cp_ob, x)
model.Minimize(cp_ob)

# Instantiate a CP solver 
cp_solver = cp_model.CpSolver()
cp_solver.parameters.max_time_in_seconds = 40.0

# Solve and compute time
start_time = time.time()
res_status = cp_solver.Solve(model)
end_time = time.time()

# Print solution
if res_status == cp_model.OPTIMAL or res_status == cp_model.FEASIBLE:
  obj_value = int(cp_solver.Value(cp_ob)) 
  solution = []
  for i in range(obj_value):
    solution.append([-1 for _ in range(M)])
  for i in range(N):
    for j in range(M):
      if cp_solver.Value(y[i][j]) == 1:
        solution[int(cp_solver.Value(x[i]) - 1)][j] = i
        break
  printSolution()
else:
  print('Not found solution')

print(f'Used time: {1000*(end_time - start_time)} milliseconds')

Not found solution
Used time: 53310.627460479736 milliseconds


## Heuristic Algorithm



### Heuristic Algorithm 1

In [5]:
print('Heuristic 1')

startTime = time.time()

conflicts = {} #conflicts[i] = list of exams that cannot be administered in the same period as exam i+1
for pair in p:
    conflicts.setdefault(pair[0], []).append(pair[1])
    conflicts.setdefault(pair[1], []).append(pair[0])

sortedRooms = sorted([(c[i], i) for i in range(M)]) #sort roomss in ascending order of capacity
result = [[None] * M] #result[i, k] = exam administered in period i+1 and room k+1
print('\nExam', 'Period', 'Room', sep='\t')
for exam in range(N): #sequentially assign a period and a room to each exam
    nextExam = False
    for period in range(len(result) + 1): #consider existing periods first
        if period == len(result):
            #if this exam cannot be held in any existing period, set up a new period
            result.append([None] * M)
        notThisPeriod = False
        if exam in conflicts:
            for otherExam in result[period]:
                if otherExam in conflicts[exam]:
                    notThisPeriod = True
                    break
            if notThisPeriod:
                continue 
        for room in range(M): #consider smaller rooms first to save bigger ones for other exams
            capacity = sortedRooms[room][0]
            roomIndex = sortedRooms[room][1]
            if result[period][roomIndex] == None and capacity >= d[exam]:
                result[period][roomIndex] = exam
                print(exam + 1, period + 1, room + 1, sep='\t') #print schedule by exam
                nextExam = True
                break
        if nextExam:
            break


#PRINT RESULT

print(f'\nUsed time: {(time.time() - startTime) * 1000} milliseconds')

numberOfDays = ceil(len(result) / 4)
print(f'\nThe number of days to administer all exams is {numberOfDays}.')

if input('\nEnter "y" to see details. ').lower() in ("y", "yes"):
    for period in range(len(result)): #print schedule by period
        if period % 4 == 0:
            print(f'Day {period // 4 + 1}:')
        print(f'\tPeriod {period + 1}:')
        for room in range(M):
            exam = result[period][room]
            conflictsOfThisExam = [e + 1 for e in conflicts.get(exam, [])]
            if exam != None:
                print(f'\t\tRoom {room + 1} (capacity = {c[room]}): Exam {exam + 1} (expected attendants = {d[exam]}, exams with common candidates = {conflictsOfThisExam})')

Heuristic 1

Exam	Period	Room
1	1	1
2	1	6
3	2	6
4	1	3
5	1	4
6	1	5
7	2	4
8	1	2
9	1	7
10	1	8
11	1	9
12	1	10
13	1	11
14	1	12
15	1	13
16	1	14
17	1	15
18	1	16
19	1	17
20	1	18
21	2	5
22	1	19
23	2	7
24	1	20
25	1	21
26	1	22
27	3	1
28	2	1
29	1	23
30	2	8
31	1	24
32	1	25
33	2	9
34	2	10
35	2	2
36	2	3
37	2	11
38	3	4
39	2	12
40	2	13
41	3	6
42	2	14
43	2	15
44	2	16
45	2	17
46	3	3
47	2	18
48	3	5
49	2	19
50	3	2
51	2	20
52	2	21
53	3	7
54	3	8
55	4	3
56	2	22
57	2	23
58	2	24
59	3	9
60	2	25
61	4	4
62	3	10
63	3	11
64	4	1
65	3	12
66	4	2
67	3	13
68	3	14
69	4	5
70	3	15
71	3	16
72	3	17
73	3	18
74	3	19
75	3	20
76	4	6
77	4	7
78	3	21
79	4	8
80	3	22
81	3	23
82	3	24
83	3	25
84	4	9
85	5	4
86	4	10
87	4	11
88	4	12
89	4	13
90	4	14
91	4	15
92	4	16
93	4	17
94	5	5
95	5	1
96	5	3
97	4	18
98	5	6
99	4	19
100	4	20
101	4	21
102	5	7
103	5	2
104	5	8
105	4	22
106	5	9
107	4	23
108	4	24
109	5	10
110	4	25
111	6	1
112	5	11
113	5	12
114	5	13
115	7	6
116	5	14
117	5	15
118	5	16
119	5	17
120	6	4
121	5	18
122	5	19
123	5	20
124	5	21
125	5	22
1

### Heuristic Algorithm 2

In [7]:
print('Heuristic 2')
conflicts = {} #conflicts[i] = list of exams that cannot be administered in the same period as exam i+1
for pairs in p:
    conflicts.setdefault(pairs[0], []).append(pairs[1])
    conflicts.setdefault(pairs[1], []).append(pairs[0])

print('\nPeriod', 'Room', 'Exam', sep='\t')

sortedExams = sorted([(d[i], i) for i in range(N)], reverse=True) #sort exams in ascending order of capacity

schedule = [] #schedule[i, k] = exam administered in period i+1 and room k+1
period = 0
start_time = time.time()
while sortedExams: #sequentially fill each period with as many exams as possible until all exams have been scheduled
    schedule.append([None] * M)
    for room in range(M):
        for exam in sortedExams: #consider more popular exams first
            if exam[0] <= c[room]: #if a hall has adequate capacity
                #check if any exam already scheduled in this period has common candidates with the one being considered
                noConflict = True
                if exam[1] in conflicts:
                    for scheduledExam in schedule[period]:
                        if scheduledExam in conflicts[exam[1]]:
                            noConflict = False
                            break
                if noConflict: #schedule exam in period and room and remove from list of exams to schedule
                    schedule[period][room] = exam[1]
                    sortedExams.remove(exam)
                    break
    period += 1
#PRINT RESULT
end_time = time.time()

print(f'\nUsed time: {(end_time - start_time) * 1000} milliseconds')

numberOfDays = ceil(period / 4)
print(f'\nThe number of days to administer all exams is {numberOfDays}.')

if input('\nEnter "y" to see details. ').lower() in ("y", "yes"):
    for pe in range(period): #print schedule by period
        if pe % 4 == 0:
            print(f'Day {pe // 4 + 1}:')
        print(f'\tPeriod {pe + 1}:')
        for room in range(M):
            exam = schedule[pe][room]
            conflictsOfThisExam = [e + 1 for e in conflicts.get(exam, [])]
            if exam != None:
                print(f'\t\tRoom {room + 1} (capacity = {c[room]}): Exam {exam + 1} (expected attendants = {d[exam]}, exams with common candidates = {conflictsOfThisExam})')

Heuristic 2

Period	Room	Exam

Used time: 2.002239227294922 milliseconds

The number of days to administer all exams is 3.

Enter "y" to see details. y
Day 1:
	Period 1:
		Room 1 (capacity = 60): Exam 163 (expected attendants = 60, exams with common candidates = [194, 132, 131])
		Room 2 (capacity = 63): Exam 161 (expected attendants = 60, exams with common candidates = [181, 53, 76, 180, 177, 144])
		Room 3 (capacity = 61): Exam 148 (expected attendants = 60, exams with common candidates = [75, 178, 171, 40, 45, 19])
		Room 4 (capacity = 39): Exam 165 (expected attendants = 39, exams with common candidates = [136, 129, 74])
		Room 5 (capacity = 63): Exam 141 (expected attendants = 60, exams with common candidates = [151, 74, 12])
		Room 6 (capacity = 63): Exam 133 (expected attendants = 59, exams with common candidates = [84, 156, 78, 12, 50, 55])
		Room 7 (capacity = 65): Exam 125 (expected attendants = 59, exams with common candidates = [11, 200, 87])
		Room 8 (capacity = 60): Exam 

### Heuristic Algorithm 3

In [8]:
print('Heuristic 3')

startTime = time.time()

conflicts = {} #conflicts[i] = list of exams that cannot be administered in the same period as exam i+1
for pairss in p:
    conflicts.setdefault(pairss[0], []).append(pairss[1])
    conflicts.setdefault(pairss[1], []).append(pairss[0])

sortedRooms = sorted([(c[i], i) for i in range(M)]) #sort rooms in ascending order of capacity
result = [[None] * M] #result[i, k] = exam administered in period i+1 and room k+1
print('\nExam', 'Period', 'Room', sep='\t')
for exam in range(N): #sequentially assign a period and a room to each exam
    stop = False
    for period in range(len(result) + 1): #consider existing periods first 
        for room in range(M): #consider smaller rooms first to save bigger ones for other exams
            capacity = sortedRooms[room][0]
            roomIndex = sortedRooms[room][1]
            if capacity >= d[exam] and result[period][roomIndex] == None:
                noConflict = True
                if exam in conflicts:
                    for otherExam in result[period]:
                        if otherExam in conflicts[exam]:
                            noConflict = False
                            break
                if noConflict:
                    result[period][roomIndex] = exam
                    print(exam + 1, period + 1, room + 1, sep='\t') #print schedule by exam
                    stop = True
                    break
        if stop:
            break
        if period == len(result) - 1:
            #if this exam cannot be held in any existing period, set up a new period
            result.append([None] * M)

#PRINT RESULT

print(f'\nUsed time: {(time.time() - startTime) * 1000} milliseconds')

numberOfDays = ceil(len(result) / 4)
print(f'\nThe number of days to administer all exams is {numberOfDays}.')

if input('\nEnter "y" to see details. ').lower() in ("y", "yes"):
    for period in range(len(result)): #print schedule by period
        if period % 4 == 0:
            print(f'Day {period // 4 + 1}:')
        print(f'\tPeriod {period + 1}:')
        for room in range(M):
            exam = result[period][room]
            conflictsOfThisExam = [e + 1 for e in conflicts.get(exam, [])]
            if exam != None:
                print(f'\t\tRoom {room + 1} (capacity = {c[room]}): Exam {exam + 1} (expected attendants = {d[exam]}, exams with common candidates = {conflictsOfThisExam})')

Heuristic 3

Exam	Period	Room
1	1	1
2	1	6
3	2	6
4	1	3
5	1	4
6	1	5
7	2	4
8	1	2
9	1	7
10	1	8
11	1	9
12	1	10
13	1	11
14	1	12
15	1	13
16	1	14
17	1	15
18	1	16
19	1	17
20	1	18
21	2	5
22	1	19
23	2	7
24	1	20
25	1	21
26	1	22
27	3	1
28	2	1
29	1	23
30	2	8
31	1	24
32	1	25
33	2	9
34	2	10
35	2	2
36	2	3
37	2	11
38	3	4
39	2	12
40	2	13
41	3	6
42	2	14
43	2	15
44	2	16
45	2	17
46	3	3
47	2	18
48	3	5
49	2	19
50	3	2
51	2	20
52	2	21
53	3	7
54	3	8
55	4	3
56	2	22
57	2	23
58	2	24
59	3	9
60	2	25
61	4	4
62	3	10
63	3	11
64	4	1
65	3	12
66	4	2
67	3	13
68	3	14
69	4	5
70	3	15
71	3	16
72	3	17
73	3	18
74	3	19
75	3	20
76	4	6
77	4	7
78	3	21
79	4	8
80	3	22
81	3	23
82	3	24
83	3	25
84	4	9
85	5	4
86	4	10
87	4	11
88	4	12
89	4	13
90	4	14
91	4	15
92	4	16
93	4	17
94	5	5
95	5	1
96	5	3
97	4	18
98	5	6
99	4	19
100	4	20
101	4	21
102	5	7
103	5	2
104	5	8
105	4	22
106	5	9
107	4	23
108	4	24
109	5	10
110	4	25
111	6	1
112	5	11
113	5	12
114	5	13
115	7	6
116	5	14
117	5	15
118	5	16
119	5	17
120	6	4
121	5	18
122	5	19
123	5	20
124	5	21
125	5	22
1

## Backtracking (Brute Force)

In [ ]:
end = 1000000
conflict = [[] for _ in range(N)]

for k in p:
  u, v = k[0], k[1]
  conflict[u].append(v)
  conflict[v].append(u)

# assign period
period = [-1] * N

# room
room = []
for _ in range(N):
  room.append([-1] * M)

def isPlaceable(u, slot):
  if period[u] >= 0:
    return False
  for v in conflict[u]:
    if period[v] == slot:
      return False
  return True

def dfs(u, slot):
  global end
  if u == N:
    end = min(end, slot)
    return
  if slot > end:
    return
  for j in range(M):
    if room[slot][j] == -1:
      for i in range(N):
        if isPlaceable(i, slot) and d[i] <= c[j]:
          period[i], room[slot][j] = slot, i
          dfs(u + 1, slot)
          period[i], room[slot][j] = -1, -1
  dfs(u, slot + 1)
  return

# Solve
start_time = time.process_time()
dfs(0, 0)
end_time = time.process_time()

# Solution
if end != 1000000:
  print(f'Objective value: {end + 1} periods')
else:
  print('No found solution.')
print('------------------')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')